In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN

In [ ]:
import pandas as pd

In [ ]:
dataset = pd.read_csv('test_new.csv')

In [ ]:
dataset.head(100)

In [ ]:
x = np.array(dataset['X'])
y = np.array(dataset['Y'])
z = np.array(dataset['Z'])
r = np.array(dataset['Red'])
g = np.array(dataset['Green'])
b = np.array(dataset['Blue'])

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
z.shape

In [ ]:
xyz=np.column_stack((x,y,z))
rgb=np.column_stack((r,g,b))


In [ ]:
xyz

In [ ]:
import open3d as o3d

In [ ]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
o3d.io.write_point_cloud("test_new_pcd.ply", pcd)

In [ ]:
# Load saved point cloud and visualize it
pcd_load = o3d.io.read_point_cloud("test_new_pcd.ply")

# Convert Open3D.o3d.geometry.PointCloud to numpy array
xyz_load = np.asarray(pcd_load.points)
print('xyz_load')
print(xyz_load)
o3d.visualization.draw_geometries([pcd_load])

In [ ]:
'''
pcd = o3d.io.read_point_cloud("test_new_pcd.ply")
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])
                                  
'''

In [ ]:
pcd = o3d.io.read_point_cloud("test_new_pcd.ply")

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd.cluster_dbscan(eps=0.02, min_points=10, print_progress=True))

max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
o3d.visualization.draw_geometries([pcd],
                                  zoom=0.455,
                                  front=[-0.4999, -0.1659, -0.8499],
                                  lookat=[2.1813, 2.0619, 2.0999],
                                  up=[0.1204, -0.9852, 0.1215])

In [ ]:
print("Testing kdtree in Open3D...")
print("Load a point cloud and paint it gray.")
pcd = o3d.io.read_point_cloud("test_new_pcd.ply")
pcd.paint_uniform_color([0.5, 0.5, 0.5])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

In [ ]:
print("Paint the 1500th point red.")
pcd.colors[150] = [1, 0, 0]

In [ ]:
print("Find its 200 nearest neighbors, and paint them blue.")
[k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[150], 200)
np.asarray(pcd.colors)[idx[1:], :] = [0, 0, 1]

In [ ]:
print("Find its neighbors with distance less than 0.2, and paint them green.")
[k, idx, _] = pcd_tree.search_radius_vector_3d(pcd.points[1500], 0.2)
np.asarray(pcd.colors)[idx[1:], :] = [0, 1, 0]

In [ ]:
print("Visualize the point cloud.")
o3d.visualization.draw_geometries([pcd])

In [ ]:
%matplotlib inline
import open3d as o3d
#o3d.visualization.webrtc_server.enable_webrtc()
if __name__ == "__main__":
    sample_ply_data = o3d.data.PLYPointCloud()
    pcd = o3d.io.read_point_cloud("test_new_pcd.ply")
    # Flip it, otherwise the pointcloud will be upside down.
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])
    print(pcd)
    axis_aligned_bounding_box = pcd.get_axis_aligned_bounding_box()
    axis_aligned_bounding_box.color = (1, 0, 0)
    oriented_bounding_box = pcd.get_oriented_bounding_box()
    oriented_bounding_box.color = (0, 1, 0)
    print(
        "Displaying axis_aligned_bounding_box in red and oriented bounding box in green ..."
    )
    o3d.visualization.draw_geometries(
        [pcd, axis_aligned_bounding_box, oriented_bounding_box])

In [ ]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt


if __name__ == "__main__":
    sample_ply_data = o3d.data.PLYPointCloud()
    pcd = o3d.io.read_point_cloud("test_new_pcd.ply")
    # Flip it, otherwise the pointcloud will be upside down.
    pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        labels = np.array(
            pcd.cluster_dbscan(eps=0.5, min_points=5, print_progress=True))

    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    #colors[labels < 0] = 0
    print(labels)
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    #o3d.visualization.webrtc_server.enable_webrtc()
    o3d.visualization.draw_geometries([pcd])
    #o3d.visualization.webrtc_server.enable_webrtc()

[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 85355
point cloud has 85355 clusters


In [ ]:
pcd = o3d.io.read_point_cloud("test_new_pcd.ply")
plane_model, inliers = pcd.segment_plane(distance_threshold=1,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [1]:
'''
import numpy as np
import open3d as o3d
from sklearn.cluster import KMeans, DBSCAN, OPTICS
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

if __name__ == '__main__':
    # Read point cloud:
    pcd = o3d.io.read_point_cloud("test_new_pcd.ply")
    o3d.visualization.draw_geometries([pcd])
    # Get points and transform it to a numpy array:
    points = np.asarray(pcd.points).copy()

    # Project points on 0XZ plane:
    points[:, 1] = 0
    pcd_projected = o3d.geometry.PointCloud()  # create point cloud object
    pcd_projected.points = o3d.utility.Vector3dVector(points)  # set pcd_np as the point cloud points
    o3d.visualization.draw_geometries([pcd_projected])

    # projection: consider the x and z coordinates:
    points_xz = points[:, [0, 2]]
    # Normalisation:
    scaled_points = StandardScaler().fit_transform(points_xz)
    # Clustering:
    model = DBSCAN(eps=0.15, min_samples=10)
    model.fit(scaled_points)

    # Get labels:
    labels = model.labels_
    # Get the number of colors:
    n_clusters = len(set(labels))

    # Mapping the labels classes to a color map:
    colors = plt.get_cmap("tab20")(labels / (n_clusters if n_clusters > 0 else 1))
    # Attribute to noise the black color:
    colors[labels < 0] = 0
    # Update points colors:
    pcd_projected.colors = o3d.utility.Vector3dVector(colors[:, :3])
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

    # Display:
    o3d.visualization.draw_geometries([pcd_projected])
    o3d.visualization.draw_geometries([pcd])
'''

'\nimport numpy as np\nimport open3d as o3d\nfrom sklearn.cluster import KMeans, DBSCAN, OPTICS\nimport matplotlib.pyplot as plt\nfrom sklearn.preprocessing import StandardScaler\n\nif __name__ == \'__main__\':\n    # Read point cloud:\n    pcd = o3d.io.read_point_cloud("test_new_pcd.ply")\n    o3d.visualization.draw_geometries([pcd])\n    # Get points and transform it to a numpy array:\n    points = np.asarray(pcd.points).copy()\n\n    # Project points on 0XZ plane:\n    points[:, 1] = 0\n    pcd_projected = o3d.geometry.PointCloud()  # create point cloud object\n    pcd_projected.points = o3d.utility.Vector3dVector(points)  # set pcd_np as the point cloud points\n    o3d.visualization.draw_geometries([pcd_projected])\n\n    # projection: consider the x and z coordinates:\n    points_xz = points[:, [0, 2]]\n    # Normalisation:\n    scaled_points = StandardScaler().fit_transform(points_xz)\n    # Clustering:\n    model = DBSCAN(eps=0.15, min_samples=10)\n    model.fit(scaled_points)\n